<a href="https://colab.research.google.com/github/zqiaohe/100daysofcode/blob/master/%5Bhomework%5Dsegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p style="align: center;"><img align=center src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" style="height:400px;" width=500px/></p>

<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

<h3 style="text-align: center;"><b>Домашнее задание: сегментация изображений</b></h3>

---

<h3 style="text-align: center;"><b>Условие</b></h3>

В качестве домашней работы предлагается улучшить сегментацию, сделанную на семинаре. В качестве основы для даталоэдера используйте функцию __keras_generator__. Попробуйте сделать следующие улучшения:
1. Брать случайный кроп, а не ресайзить исходное изображение. Попробуйте разные размеры кропов, на каких результат получается лучше? Кроп какого максимального размера помещается в вашу видеокарту?
2. Добавить аугментаций. Например, можно поворачивать картинки или добавлять случайный шум

Эти два пункта улучшения можно сделать с помощью библиотеки __albumentations__ https://github.com/albu/albumentations . Пример использования можно посмотреть в конце семинарского ноутбука.

In [0]:
import numpy as np
import cv2
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!unzip /content/gdrive/My\ Drive/[11]segmentation/data

In [0]:
def rle_decode(mask_rle, shape=(1280, 1918, 1)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)

    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths    
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
        
    img = img.reshape(shape)
    return img

In [0]:
from albumentations import (
    CLAHE, RandomRotate90, Transpose, RandomCrop, Resize, ShiftScaleRotate, Blur, OpticalDistortion, 
    GridDistortion, HueSaturationValue, IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, 
    MedianBlur, IAAPiecewiseAffine, IAASharpen, IAAEmboss, RandomContrast, RandomBrightness, 
    Flip, HorizontalFlip, OneOf, Compose, PadIfNeeded, LongestMaxSize, PadIfNeeded, ElasticTransform,Cutout
)




def strong_aug(p=1.0):
    return Compose([
        ShiftScaleRotate(shift_limit=0.125, scale_limit=0.2, rotate_limit=10, p=0.7, border_mode=cv2.BORDER_CONSTANT),
        RandomCrop(256, 256),
        #PadIfNeeded(min_height=224, min_width=224, border_mode=cv2.BORDER_CONSTANT, p=1.0),
        #Resize(64, 64),
        RandomRotate90(),
        ElasticTransform(1.), 
        HorizontalFlip(),
        #Cutout(p=1.),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.3),
        OneOf([
            MotionBlur(p=.4),
            MedianBlur(blur_limit=3, p=0.3),
            Blur(blur_limit=3, p=0.3),
        ], p=0.5),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=0.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.5),
        OneOf([
            CLAHE(clip_limit=3),
            IAASharpen(),
            IAAEmboss(),
            RandomContrast(),
            RandomBrightness(),
        ], p=0.4),
        HueSaturationValue(p=0.7),
         
        
        
    ],
        p=p)

In [0]:
df = pd.read_csv('data/train_masks.csv')

In [0]:
train_df = df[:4000]
val_df = df[4000:]

In [0]:
def keras_generator(gen_df, batch_size, aug):
    while True:
        x_batch = []
        y_batch = []
        
        for i in range(batch_size):
            img_name, mask_rle = gen_df.sample(1).values[0]
            img = cv2.imread('data/train/{}'.format(img_name))
            mask = rle_decode(mask_rle)
            if aug == 0:
              img = cv2.resize(img, (256, 256))
              mask = cv2.resize(mask, (256, 256))
            else:
              augmentation = strong_aug(p=1.0)
              data = {'image': img.astype('uint8'), 'mask': mask.astype('uint8')}
              augmented = augmentation(**data)
              img, mask = augmented["image"], augmented["mask"]
            
            x_batch += [img]
            y_batch += [mask]

        x_batch = np.array(x_batch) / 255.
        y_batch = np.array(y_batch)
        
        if aug == 1:
          yield x_batch, y_batch
        else:
          yield x_batch, np.expand_dims(y_batch, -1)

Подумайте, нужно ли применять аугментации для валидационной выборки или это стоит делать только для тренировки? Возможно, следует добавить в аргументы __keras_generator__ флаг, который будет говорить, применять аугментации или нет.
3. В качестве модели машинного обучения предлагается взять либо модель с семинара, либо какую-нибудь готовую реализацию, например из __segmentation_models__ https://github.com/qubvel/segmentation_models . Пример использования:

In [8]:
!pip install git+https://github.com/qubvel/segmentation_models

  Cloning https://github.com/qubvel/segmentation_models to /tmp/pip-req-build-q8wd6_yh
  Running setup.py bdist_wheel for segmentation-models ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-x0yvc3ry/wheels/49/cf/46/cbb4bb64518c402aea99df9d466f1081450597e653256bbcf4
Successfully built segmentation-models


In [0]:
batch_size = 16

In [10]:
from segmentation_models import Unet
from segmentation_models.backbones import get_preprocessing

for x, y in keras_generator(train_df, 10, 1):
    break
preprocessing_fn = get_preprocessing('resnet34')
x = preprocessing_fn(x)

# prepare model
model = Unet(backbone_name='densenet121', encoder_weights='imagenet')
model.compile('Adam', 'binary_crossentropy', ['binary_accuracy'])

Using TensorFlow backend.


29089792/29084464 [==============================] - 2s 0us/step


In [11]:
model.fit(x,y,epochs = 10)

Epoch 1/10
10/10 [==============================] - 31s 3s/step - loss: 0.7968 - binary_accuracy: 0.4243
Epoch 2/10
10/10 [==============================] - 1s 66ms/step - loss: 0.6791 - binary_accuracy: 0.5688
Epoch 3/10
10/10 [==============================] - 1s 66ms/step - loss: 0.5483 - binary_accuracy: 0.7594
Epoch 4/10
10/10 [==============================] - 1s 65ms/step - loss: 0.4435 - binary_accuracy: 0.9017
Epoch 5/10
10/10 [==============================] - 1s 65ms/step - loss: 0.3798 - binary_accuracy: 0.9748
Epoch 6/10
10/10 [==============================] - 1s 66ms/step - loss: 0.3288 - binary_accuracy: 0.9679
Epoch 7/10
10/10 [==============================] - 1s 65ms/step - loss: 0.2887 - binary_accuracy: 0.9895
Epoch 8/10
10/10 [==============================] - 1s 65ms/step - loss: 0.2628 - binary_accuracy: 0.9883
Epoch 9/10
10/10 [==============================] - 1s 65ms/step - loss: 0.2437 - binary_accuracy: 0.9941
Epoch 10/10
10/10 [============================

In [0]:
preprocessing_fn = get_preprocessing('vgg16')
x = preprocessing_fn(x)

# prepare model
model = Unet(backbone_name='vgg16', encoder_weights='imagenet')
model.compile('Adam', 'binary_crossentropy', ['binary_accuracy'])

In [13]:
model.fit(x,y,epochs = 10)

Epoch 1/10
10/10 [==============================] - 15s 2s/step - loss: 0.7057 - binary_accuracy: 0.1864
Epoch 2/10
10/10 [==============================] - 1s 67ms/step - loss: 0.6733 - binary_accuracy: 0.8467
Epoch 3/10
10/10 [==============================] - 1s 67ms/step - loss: 0.6498 - binary_accuracy: 0.8505
Epoch 4/10
10/10 [==============================] - 1s 66ms/step - loss: 0.6374 - binary_accuracy: 0.8521
Epoch 5/10
10/10 [==============================] - 1s 66ms/step - loss: 0.6287 - binary_accuracy: 0.8505
Epoch 6/10
10/10 [==============================] - 1s 66ms/step - loss: 0.6230 - binary_accuracy: 0.8531
Epoch 7/10
10/10 [==============================] - 1s 66ms/step - loss: 0.6173 - binary_accuracy: 0.8554
Epoch 8/10
10/10 [==============================] - 1s 66ms/step - loss: 0.6123 - binary_accuracy: 0.8554
Epoch 9/10
10/10 [==============================] - 1s 66ms/step - loss: 0.6072 - binary_accuracy: 0.8554
Epoch 10/10
10/10 [============================

В качестве базового энкодера на семинаре использовался __resnet50__. Поэкспериментируйте с разными энкодерами, например, __VGG16__, __densenet121__. Какой дает лучшее качество?

Итак, вы обучили модель предсказывать кропы, например, размера 256x256, качество мерили тоже на кропах. Но как исходная цель - нужно уметь делать предсказания на всю картинку. Как решить проблему и есть ли она вообще? Первый приходящий в голову вариант - можно проходиться по картинке окном в 256x256 пикселей и сохранять предсказания на всю большую картинку. Но можно сделать проще, если ответить на вопрос: работает ли полностью сверточная сеть на картинках произвольного размера(с добиванием до кратности паддингами)? Почему не работали сети для классификации?

4. Напоследок, можете попробовать технику под названием __Test Time Augmentation__. Сделайте предсказание картинки, сохраните его. Попробуйте сделать flip картинки, и предсказать флипнутое изображение. Что будет если усреднить результаты этих двух предсказаний? Улучшится ли качество?

---

<h3 style="text-align: center;"><b>Ваше решение</b></h3>